In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, precision_score, recall_score, f1_score

# Load your dataset
data = pd.read_csv('/content/sample_data/vibration_data.csv')

# Extract features and labels
X = data[['ROLL_BAD', 'PITCH_BAD']].values
y = data[['ROLL_FILTERED', 'PITCH_FILTERED']].values

# Normalize the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

# Split the data into training, validation, and test sets (60%, 20%, 20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Reshape the data to fit the model (adding a channel dimension)
X_train = X_train.reshape(-1, X_train.shape[1], 1)
X_val = X_val.reshape(-1, X_val.shape[1], 1)
X_test = X_test.reshape(-1, X_test.shape[1], 1)


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Define the model
model = Sequential([
    Conv1D(32, kernel_size=1, activation='relu', input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    Conv1D(16, kernel_size=1, activation='relu'),
    Dropout(0.2),
    Flatten(),
    Dense(2)  # Output layer with 2 neurons for ROLL_FILTERED and PITCH_FILTERED
])

# Compile the model
model.compile(optimizer=Adam(), loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/50
57/57 [==============================] - 1s 6ms/step - loss: nan - val_loss: nan
Epoch 2/50
57/57 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 3/50
57/57 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 4/50
57/57 [==============================] - 0s 4ms/step - loss: nan - val_loss: nan
Epoch 5/50
57/57 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 6/50
57/57 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 7/50
57/57 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 8/50
57/57 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 9/50
57/57 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 10/50
57/57 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 11/50
57/57 [==============================] - 0s 3ms/step - loss: nan - 

In [3]:
# Make predictions
y_pred = model.predict(X_test)

# Handle NaN values in predictions by replacing them with zero or a specific value
y_pred = np.nan_to_num(y_pred)

# Inverse transform the predictions and actual values to their original scale
y_pred = scaler_y.inverse_transform(y_pred)
y_test = scaler_y.inverse_transform(y_test)

# Evaluate the model in terms of regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'R-squared (R2): {r2:.4f}')


19/19 [==============================] - 1s 4ms/step
Mean Squared Error (MSE): 339.3893
Mean Absolute Error (MAE): 14.1607
R-squared (R2): -0.0032


In [5]:
# Define the bins for each dimension separately
bins_roll = np.linspace(start=min(y_test[:, 0].min(), y_pred[:, 0].min()), stop=max(y_test[:, 0].max(), y_pred[:, 0].max()), num=10)
bins_pitch = np.linspace(start=min(y_test[:, 1].min(), y_pred[:, 1].min()), stop=max(y_test[:, 1].max(), y_pred[:, 1].max()), num=10)

# Digitize the actual and predicted values into bins
y_test_binned_roll = np.digitize(y_test[:, 0], bins_roll)
y_pred_binned_roll = np.digitize(y_pred[:, 0], bins_roll)

y_test_binned_pitch = np.digitize(y_test[:, 1], bins_pitch)
y_pred_binned_pitch = np.digitize(y_pred[:, 1], bins_pitch)

# Evaluate the model in terms of accuracy, precision, recall, and F-Score for ROLL
accuracy_roll = accuracy_score(y_test_binned_roll, y_pred_binned_roll)
precision_roll = precision_score(y_test_binned_roll, y_pred_binned_roll, average='macro', zero_division=0)
recall_roll = recall_score(y_test_binned_roll, y_pred_binned_roll, average='macro')
f1_roll = f1_score(y_test_binned_roll, y_pred_binned_roll, average='macro')

print(f'ROLL - Accuracy: {accuracy_roll:.4f}')
print(f'ROLL - Precision: {precision_roll:.4f}')
print(f'ROLL - Recall: {recall_roll:.4f}')
print(f'ROLL - F1-Score: {f1_roll:.4f}')

# Evaluate the model in terms of accuracy, precision, recall, and F-Score for PITCH
accuracy_pitch = accuracy_score(y_test_binned_pitch, y_pred_binned_pitch)
precision_pitch = precision_score(y_test_binned_pitch, y_pred_binned_pitch, average='macro', zero_division=0)
recall_pitch = recall_score(y_test_binned_pitch, y_pred_binned_pitch, average='macro')
f1_pitch = f1_score(y_test_binned_pitch, y_pred_binned_pitch, average='macro')

print(f'PITCH - Accuracy: {accuracy_pitch:.4f}')
print(f'PITCH - Precision: {precision_pitch:.4f}')
print(f'PITCH - Recall: {recall_pitch:.4f}')
print(f'PITCH - F1-Score: {f1_pitch:.4f}')


ROLL - Accuracy: 0.1689
ROLL - Precision: 0.0169
ROLL - Recall: 0.1000
ROLL - F1-Score: 0.0289
PITCH - Accuracy: 0.3179
PITCH - Precision: 0.0318
PITCH - Recall: 0.1000
PITCH - F1-Score: 0.0482
